## Game plan!

How it will work: 
    1. We use Spotify API to gather top study playlists somehow into a giant database --use "get several tracks" to get data abt all songs in the playlist  
    2. We cluster the songs by their attributes and use elbow method to determine clusters  
    3. We create categories of the clusters by analyzing its top genres?   
    4. Personality test for students to get their ideal playlist!  

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

### Step 1: Collect playlists
Step 1: Collect playlists
We should have a final dataframe where each row is a song and the columns are: song id, artists id, danceability, energy, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration, count

In [38]:
cid = 'af25a5ab6fac412a9f644871d866df12' 
secret = '374a71348a9943ba8b585b9c9466feb6'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout = 10, retries = 10)



### Get a massive list of non-Spotify playlists

In [10]:
'''
takes in playlist id and the number of tracks on it
for each iteration, it finds id and artist information for 50 songs on the playlist
returns a list of ids
'''
def get_track_ids(playlist_id, num_tracks):
# iterate through track ids in the playlist to create giant list of track ids
    ids = []
    limit = 50
    
    for i in range(0, num_tracks, limit):
        sub_playlist = sp.playlist_items(playlist_id, fields='items(track(id, name))', market= 'us', limit=limit, offset=i) 
        
        # check for None type
        tracks = [track for track in sub_playlist['items'] if track['track'] is not None]
        
        for i, tr in enumerate(tracks): 
            ids.append(tr['track']['id'])
    
    return ids

In [8]:
# create final dataframe
columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
           'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 
           'time_signature']
study_tracks = pd.DataFrame(columns=columns)

# create sentiment analyzer object 
sid = SentimentIntensityAnalyzer()

# variables
playlist_id = []
num_tracks = []
num_playlists = 900 # analyze nine hundred study playlists (this is max available)
playlist_limit = 50
description = []
sentiment = []
spotify_study = 0

# collect playlist IDs
for i in range(0,num_playlists,playlist_limit):
    playlist_results = sp.search(q='study', type='playlist', market= 'us', limit=playlist_limit,offset=i) 
    for i, pl in enumerate(playlist_results['playlists']['items']):
    # don't want spotify music
        if pl['owner']['id'] == 'spotify': 
            spotify_study+=1
            continue
        else:
            # get playlist ID and num tracks 
            playlist_id.append(pl['uri'].split(':')[2]) 
            num_tracks.append(pl['tracks']['total'])
            descript = pl['description']
            description.append(descript)
            sentiment.append(sid.polarity_scores(descript)['compound'])
print('Done Collecting Playlist IDs!')
print('Number of Spotify Playlists ommitted: ', spotify_study)
print('Number of Playlists collected: ', len(playlist_id))

# create playlist dataframe
playlist_df = pd.DataFrame({'playlist_id': playlist_id, 'num_tracks': num_tracks, 'description': description,
                           'sentiment': sentiment})

Done Collecting Playlist IDs!
Number of Spotify Playlists ommitted:  99
Number of Playlists collected:  801


In [11]:
track_ids = []
num_playlists_done = 0

# remove duplicate playlists 
playlist_df = playlist_df.groupby('playlist_id').max().reset_index()

# update values
playlist_id = playlist_df['playlist_id'].to_list()
num_tracks = playlist_df['num_tracks'].to_list()
print('Collecting tracks for ', len(playlist_id), ' playlists!')

# add each track's audio features onto study tracks
for i in range(len(playlist_id)):
    if num_playlists_done % 25 == 0:
        print(num_playlists_done, ' done!')
        print('continuing...')
        
    # extract track ids from each playlist
    track_ids = track_ids + get_track_ids(playlist_id[i], num_tracks[i])
    
    num_playlists_done+=1

print('Done collecting track IDs!')

0  done!
continuing...
25  done!
continuing...
50  done!
continuing...
75  done!
continuing...
100  done!
continuing...
125  done!
continuing...
150  done!
continuing...
175  done!
continuing...
200  done!
continuing...
225  done!
continuing...
250  done!
continuing...
275  done!
continuing...
300  done!
continuing...
325  done!
continuing...
350  done!
continuing...
375  done!
continuing...
400  done!
continuing...
425  done!
continuing...
450  done!
continuing...
475  done!
continuing...
500  done!
continuing...
525  done!
continuing...
550  done!
continuing...
575  done!
continuing...
600  done!
continuing...
625  done!
continuing...
650  done!
continuing...
675  done!
continuing...
700  done!
continuing...
725  done!
continuing...
750  done!
continuing...
775  done!
continuing...
Done collecting track IDs!


In [27]:
print('number of track ids: ', len(track_ids))

# find number of unique tracks:
unique_tracks = pd.DataFrame({'track_id':track_ids})
unique_tracks = unique_tracks.value_counts().reset_index()
unique_tracks.columns = ['track_id', 'count']
unique_tracks

print('number of unique tracks: ', unique_tracks.shape[0])
unique_tracks.head()

number of track ids:  185255
number of unique tracks:  118304


track_id  count
0  4UaGQ2D4MBuRBWumZO52Ma     89
1  0V5cvmTKsYmF5FmGGEAfmS     66
2  7eo2HVkRxrDU3bQPwEiIBT     64
3  0yc6Gst2xkRu0eMLeRMGCX     56
4  1BncfTJAWxrsxyT9culBrj     55

In [39]:
unique_track_ids = unique_tracks['track_id'].to_list()

print('Studying audio features for ', len(unique_track_ids), ' tracks!')

track_limit = 100
tracks_analyzed = 0

# use track id list to add audio features to study_tracks
for i in range(0, len(unique_track_ids), track_limit): 
    id_chunk = track_ids[i:i+track_limit] 
    # filter None
    id_chunk = [element for element in id_chunk if element is not None] 
    tracks_features = sp.audio_features(tracks=id_chunk) 
    
    # filter None
    tracks_features = [element for element in tracks_features if element is not None] 
    mini_tracks_df = pd.DataFrame(tracks_features)
    study_tracks = pd.concat([study_tracks, mini_tracks_df])
    
    tracks_analyzed += len(tracks_features)


Studying audio features for  118304  tracks!
Done with:  108000 !
...


In [46]:
# merge occurance of each track 
study_tracks = pd.merge(study_tracks, unique_tracks, left_on = 'id', right_on='track_id').sort_values(by='count', ascending=False)
study_tracks

danceability  energy key  loudness mode  speechiness  acousticness  \
123763        0.6450  0.8490   8    -3.198    0       0.2000         0.226   
123762        0.6450  0.8490   8    -3.198    0       0.2000         0.226   
123771        0.6450  0.8490   8    -3.198    0       0.2000         0.226   
123770        0.6450  0.8490   8    -3.198    0       0.2000         0.226   
123769        0.6450  0.8490   8    -3.198    0       0.2000         0.226   
...              ...     ...  ..       ...  ...          ...           ...   
91566         0.0696  0.0831  10   -19.418    1       0.0447         0.965   
36177         0.1810  0.0459   2   -25.083    1       0.0386         0.967   
36178         0.1810  0.0459   2   -25.083    1       0.0386         0.967   
91563         0.4410  0.1460   7   -21.754    0       0.0368         0.761   
142257        0.0752  0.0689   7   -23.116    0       0.0492         0.960   

        instrumentalness  liveness  valence    tempo            type  \
123763             0.000    0.1040   0.8550  165.008  audio_features   
123762             0.000    0.1040   0.8550  165.008  audio_features   
123771             0.000    0.1040   0.8550  165.008  audio_features   
123770             0.000    0.1040   0.8550  165.008  audio_features   
123769             0.000    0.1040   0.8550  165.008  audio_features   
...                  ...       ...      ...      ...             ...   
91566              0.747    0.1110   0.0345   58.675  audio_features   
36177              0.915    0.1790   0.0584   91.782  audio_features   
36178              0.915    0.1790   0.0584   91.782  audio_features   
91563              0.541    0.2270   0.0629  119.975  audio_features   
142257             0.929    0.0882   0.0321   74.222  audio_features   

                            id                                   uri  \
123763  4UaGQ2D4MBuRBWumZO52Ma  spotify:track:4UaGQ2D4MBuRBWumZO52Ma   
123762  4UaGQ2D4MBuRBWumZO52Ma  spotify:track:4UaGQ2D4MBuRBWumZO52Ma   
123771  4UaGQ2D4MBuRBWumZO52Ma  spotify:track:4UaGQ2D4MBuRBWumZO52Ma   
123770  4UaGQ2D4MBuRBWumZO52Ma  spotify:track:4UaGQ2D4MBuRBWumZO52Ma   
123769  4UaGQ2D4MBuRBWumZO52Ma  spotify:track:4UaGQ2D4MBuRBWumZO52Ma   
...                        ...                                   ...   
91566   3QY0L2G2t15KFJ23yGokIl  spotify:track:3QY0L2G2t15KFJ23yGokIl   
36177   0CUGJRSyfw9i2Wi6HCF3Ct  spotify:track:0CUGJRSyfw9i2Wi6HCF3Ct   
36178   0CUGJRSyfw9i2Wi6HCF3Ct  spotify:track:0CUGJRSyfw9i2Wi6HCF3Ct   
91563   36WyxaGS7hm4hJHqUBOSUG  spotify:track:36WyxaGS7hm4hJHqUBOSUG   
142257  3uVPdCnO0tdVx2B5iVxyZq  spotify:track:3uVPdCnO0tdVx2B5iVxyZq   

                                               track_href  \
123763  https://api.spotify.com/v1/tracks/4UaGQ2D4MBuR...   
123762  https://api.spotify.com/v1/tracks/4UaGQ2D4MBuR...   
123771  https://api.spotify.com/v1/tracks/4UaGQ2D4MBuR...   
123770  https://api.spotify.com/v1/tracks/4UaGQ2D4MBuR...   
123769  https://api.spotify.com/v1/tracks/4UaGQ2D4MBuR...   
...                                                   ...   
91566   https://api.spotify.com/v1/tracks/3QY0L2G2t15K...   
36177   https://api.spotify.com/v1/tracks/0CUGJRSyfw9i...   
36178   https://api.spotify.com/v1/tracks/0CUGJRSyfw9i...   
91563   https://api.spotify.com/v1/tracks/36WyxaGS7hm4...   
142257  https://api.spotify.com/v1/tracks/3uVPdCnO0tdV...   

                                             analysis_url duration_ms  \
123763  https://api.spotify.com/v1/audio-analysis/4UaG...      180560   
123762  https://api.spotify.com/v1/audio-analysis/4UaG...      180560   
123771  https://api.spotify.com/v1/audio-analysis/4UaG...      180560   
123770  https://api.spotify.com/v1/audio-analysis/4UaG...      180560   
123769  https://api.spotify.com/v1/audio-analysis/4UaG...      180560   
...                                                   ...         ...   
91566   https://api.spotify.com/v1/audio-analysis/3QY0...      153907   
36177   https://api.spot